### ML Engineering

In [ ]:
from omegaconf import OmegaConf

In [ ]:
config = OmegaConf.load("")

In [ ]:
from functools import lru_cache

In [ ]:
@lru_cache()
def expensive_function():
    pass

In [ ]:
from omegaconf import OmegaConf

In [ ]:
from evidently.report import Report

In [ ]:
from evidently.metric_preset import DataDriftPreset

In [ ]:
report = Report(metrics=[
    DataDriftPreset()
])

In [ ]:
report.run(
    reference_data=reference,
    current_data=current
)

### AI

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
x3 = torch.roll(x, shifts=1, dim=1)

In [ ]:
import transformers

In [ ]:
from transformers import DataCollatorForLanguageModeling

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=)

In [ ]:
task = torch.distributed.broadcast(x, src=0, async_op=True)

In [ ]:
task.wait()

In [ ]:
torch.distributed.broadcast(x, src=0, async_op=True)

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
def compute_score(attention_pattern, target_pattern):
    score = attention_pattern * target_pattern
    return score.sum() / attention_pattern.sum(dim=-1)

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
batch_idx = 0

In [ ]:
scores = torch.zeros(n_layers, n_heads)

In [ ]:
for layer_idx in range(n_layers):
    hook_name = f"block.{layer_idx}.attn.hook_pattern"
    layer_attention_pattern = cache[hook_name]
    for head_idx in range(n_heads):
        attention_pattern = layer_attention_pattern[batch_idx, head_idx, :, :]
        score = compute_score(attention_pattern, target_pattern)
        scores[layer_idx, head_idx] = score

In [ ]:
import numpy as np
from gym import Env

In [ ]:
step 1: parameter partrition
step 2: move to ranks
step 3: move to device
step 4: init local optimizer
step 5: step
step 6: broadcast
step 7: sync

In [ ]:
tensor_model_parallel_size = 2
pipeline_model_parallel_size = 4

In [ ]:
num_pipeline_model_parallel_groups = 4

In [ ]:
for i in range(pipeline_model_parallel_size):
    start_rank = i*num_pipeline_model_parallel_groups
    end_rank = (i+1)*num_pipeline_model_parallel_groups
    
    for j in range(tensor_model_parallel_size):
        ranks = list(range(
            start_rank+j,
            end_rank,
            tensor_model_parallel_size
        ))
        
        print(ranks)

[0, 2]
[1, 3]
[4, 6]
[5, 7]
[8, 10]
[9, 11]
[12, 14]
[13, 15]


In [ ]:
def compute_cross_entropy(logits, targets):
    log_probs = F.log_probs(logits, dim=-1)
    return -log_probs[torch.arange(targets.shape[-1]), targets].mean()

In [ ]:
from ei

In [ ]:
step 1: partrition
step 2: rank
step 3: device
step 4: local optimizer
step 5: step
step 6: broadcast
step 7: sync

In [ ]:
import numpy as np
from gym import Env
from gym.spaces import Discrete, Box

In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        super().__init__()
        self.action_space = Discrete(3)
        self.observation_space = Box(
            low=np.array([0]),
            high=np.array([100])
        )
    
    def step(self, action):
        self.state += action - 1
        self.shower_lenth -= 1
        
        reward 

In [ ]:
from einops import einsum

In [ ]:
interference = einsum(
    W,
    W,
    "b f1 d, b f2 d -> b f1 f2"
)

In [ ]:
import math

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, d_head):
        super().__init__()
        self.d_head = d_head
    
    def forward(self, q, k, v, mask=None):
        k = k.tranpose(3, 2)
        qk = torch.matmul(q, k)
        scores = qk / math.sqrt(self.d_head)
        
        if mask != None:
            scores.mask_fill(mask==0, 1e-9)
        
        attention_weights = F.softmax(scores)
        output = torch.matmul(attention_weights, v)
        return output, attention_weights

In [ ]:
class Optimizer:
    def __init__(self, params, lr):
        self.params = params
        self.lr = lr
    
    def step(self):
        with torch.no_grad():
            for param in self.params:
                if param.requires_grad:
                    param.data -= param.grad * self.lr

In [ ]:
class BatchNorm(nn.Module):
    def __init__(self, mom, eps):
        super().__init__()
        self.mom = mom
        self.eps = eps
        self.adds = nn.Parameter(torch.zeros(1))
        self.mults = nn.Parameter(torch.ones(1))
        self.mean = nn.Parameter(torch.zeros(1))
        self.var = nn.Parameter(torch.ones(1))
    
    def update_stats(self, x):
        mean, var = x.mean(dim=-1), x.var(dim=-1)
        
        self.mean.lerp_(mean, self.mom)
        self.var.lerp_(var, self.mom)
        
        return mean, var
    
    def forward(self, x):
        with torch.no_grad():
            mean, var = self.update_stats(x)
        
        x = (x-mean)/(var+self.eps).sqrt()
        x = self.adds + self.mults*x
        return x

In [ ]:
from gym.spaces import Discrete, Box

In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        super().__init__()
        self.action_space = Discrete(3)
        self.observation_space = Box(
            low=np.array([0]),
            high=np.array([100])
        )
    
    def step(self, action):
        self.temperature -= action
        self.shower_length -= 1
        reward = 1 if 37 <= self.temperature <= 39 else 0
        done = True if self.shower_length == 0 else False
        info = {}
        
        return self.temperature, reward, done, info
    
    def reset(self):
        self.temperature = 20
        self.shower_length = 60